## Ethical AI Model Evaluation Using the UCI Adult Dataset

##### By Katherine Ebadi

In [1]:
#Imports
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import joblib

In [2]:
#.py Modules
from model_utils import load_model
from fairness import compute_fairness_score
from bias import analyze_bias_distribution
from transparency import generate_shap_values, compute_transparency_score
from privacy import estimate_privacy_risk
from consent import check_consent_traceability
from scoring import calculate_ethics_score

### Data Description

This project uses the UCI Adult (Census Income) dataset. It is a widely used resource in machine learning for classification tasks. It was extracted from the US Census databate (1994) by Barry Decker and Ronny Kohavi, with the goal of predicting whether an individual's annual income exceeds $50k based on demographic and employment-related attributes.

### Data Collection Strategy
Retrieved programmatically through sklearn.datasets.fetch_openml('adult', version=2, as_frame=True)

Target Variable: class - binary (<= 50k or >50k)

### Data Preprocessing

In [3]:
#Load UCI dataset
data = fetch_openml('adult', version=2, as_frame=True)
df = data.frame

#Clean: replace '?' with NaN and drop incomplete rows
df = df.replace('?', pd.NA).dropna()
#Ensures consistent handling of missing or unknown categorical values to avoid introducing bias in training.

df['age_group'] = pd.cut(df['age'], bins=[0, 30, 50, 100], labels=['young', 'middle', 'old'])

#Encode categorical features
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le
#Transforms text-based categories into numeric form, enabling model interpretability and compatibility.

#Split features and label
X = df.drop('class', axis=1)
y = df['class']

#Create a demographic group
demographics = X['age_group']

#Draop age group from training features
X = X.drop(columns=['age_group'])
#Enables group-based fairness evaluation. 

**Challenges:**

Missing Values are represented as '?'
Several categorical columns require encoding for compatiabilty
Imbalanced class distribution may have bias for "fairness" metric

In [4]:
print(X.dtypes)

age               int64
workclass         int32
fnlwgt            int64
education         int32
education-num     int64
marital-status    int32
occupation        int32
relationship      int32
race              int32
sex               int32
capital-gain      int64
capital-loss      int64
hours-per-week    int64
native-country    int32
dtype: object


### Data Storage and Organization
- Data stored in .csv format for testing
- Trained model.pkl using joblib
- All preprocessing and training steps are in this main.ipynb for transparent execution and for ease of following along
                                                                                 
### Data Security
- All data is public and anonymize

### Tools and Platforms
**Documentation:** Jupyter Notebook for narrative and visual integration 

**Version Control:** GitHub for source code and documentation tracking 

**Storage:** Local for easy large file management and backups

### Model Choice
**RandomForestClassifier**

- Handles both numerical and categorical variables effectively
- Reduces overfitting with ensembble averaging
- Provides feature importance score for transparancy analysis

In [5]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#Save model
joblib.dump(model, 'model.pkl')

['model.pkl']

### Model Pipeline Overview/Flowchart

Load UCI Adult Dataset -> Clean and Encode Data -> Split Data (Train/Test) -> Train Random Forest Model -> Evaluate Ethics Metrics -> Compute Metrics (Fairness, Bias, Transparency, Privacy, Consent) -> Final Ethical Score Output

### Implementation Challenges + Solutions

**Challenge:** Solution

**Large dataset:** Sample subsets and distribute SHAP calculations

**Fairness metric selection:** Multiple faireness metrics (demographics)

**Model Interpretability:** Use SHAP and feature importance plots. Various visuals to help interpretation

In [6]:
model = load_model('model.pkl')

#Create age_group for demographics (help mitigate bias for fairness)
X_test_with_age = X_test.copy()
X_test_with_age['age_group'] = pd.cut(
    X_test_with_age['age'],
    bins=[0, 30, 50, 100],
    labels=['young', 'middle', 'old']
)
demographics = X_test_with_age['age_group']

#Drop age_group before model
X_test_features = X_test_with_age.drop(columns=['age_group'])

### Evaluation Metrics

In [8]:
fairness = compute_fairness_score(model, X_test, y_test, demographics)
bias = analyze_bias_distribution(model, X_test, y_test)
sampled_X = X_test_features.sample(n=100, random_state=42)
shap_values = generate_shap_values(model, sampled_X)
transparency = compute_transparency_score(shap_values)
privacy = estimate_privacy_risk(X_test)
consent_logs = pd.read_csv('logs.csv')
consent = check_consent_traceability(consent_logs)

In [9]:
print('SHAP type:', type(shap_values))
print('SHAP sample:', shap_values[:1])

SHAP type: <class 'shap._explanation.Explanation'>
SHAP sample: .values =
array([[[-0.06092474,  0.06092474],
        [-0.00317924,  0.00317924],
        [-0.0026897 ,  0.0026897 ],
        [ 0.09499111, -0.09499111],
        [ 0.11487464, -0.11487464],
        [-0.04653284,  0.04653284],
        [-0.03729433,  0.03729433],
        [-0.05070596,  0.05070596],
        [-0.00267443,  0.00267443],
        [-0.01871989,  0.01871989],
        [ 0.03361117, -0.03361117],
        [ 0.00922187, -0.00922187],
        [-0.05851603,  0.05851603],
        [-0.00280031,  0.00280031]]])

.base_values =
array([[0.7513387, 0.2486613]])

.data =
array([[    63,      2, 106141,      5,      4,      2,     11,      0,
             4,      1,      0,      0,     45,     38]], dtype=int64)


In [10]:
print('Fairness:', fairness)
print('Bias:', bias)
print('Transparency:', transparency)
print('Privacy:', privacy)
print('Consent:', consent)

Fairness: {'young': 0.0387164283222881, 'middle': 0.27750665483584736, 'old': 0.27245508982035926}
Bias: {'prediction_mean': 0.20088446655610834, 'true_mean': 0.2435599778883361, 'bias_score': 0.04267551133222777}
Transparency: 0.823723867254886
Privacy: 0.0045662100456621
Consent: 1.0


In [11]:
logs = pd.DataFrame({'consent_id': [f'cid_{i}' for i in range(len(X_test))]})
logs.to_csv('logs.csv', index=False)

consent_logs = pd.read_csv('logs.csv')
consent = check_consent_traceability(consent_logs)

### Score Aggregation

In [12]:
scores = {
    'fairness': sum(fairness.values()) / len(fairness),
    'bias': 1 - bias['bias_score'],
    'transparency': transparency,
    'privacy': 1 - privacy,
    'consent': consent
}

final_score = calculate_ethics_score(scores)

In [13]:
for k, v in scores.items():
    print(f"{k}: {v} (type: {type(v)})")

fairness: 0.19622605765949822 (type: <class 'numpy.float64'>)
bias: 0.9573244886677722 (type: <class 'numpy.float64'>)
transparency: 0.823723867254886 (type: <class 'float'>)
privacy: 0.995433789954338 (type: <class 'float'>)
consent: 1.0 (type: <class 'float'>)


### Output

In [14]:
print('Individual Scores:')
for k, v in scores.items():
    print(f'{k.capitalize()}: {round(v, 4)}')

print('\nFinal Ethical Score (0–5):', final_score)

Individual Scores:
Fairness: 0.1962
Bias: 0.9573
Transparency: 0.8237
Privacy: 0.9954
Consent: 1.0

Final Ethical Score (0–5): 3.97


<b>References:</b>

Becker, B. & Kohavi, R. (1996). Adult [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.